In [14]:
# Install dependencies
%pip install --upgrade pip 
%pip install pandas requests

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd
import requests
import json
import os
import hashlib
import datetime
from urllib.parse import urlparse

In [ ]:
input_url = ["http://131.0.1.19:3002/postgres2/ultimas-x-horas-nuevo/24","http://131.0.1.19:3002/postgres2/datos","http://131.0.1.19:3002/postgres2/ultimas-x-horas-antiguo/24"]
post_url = ["http://131.0.1.19:3002/postgres2/datos-diarios-nuevo-2024/7","http://131.0.1.19:3002/postgres2/datos-diarios-antiguo/7"]
data_dir = "../data"

In [17]:
def save_data(json_data, filename):
    with open(os.path.join(data_dir, filename), "w") as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)
    print(f"Data saved to {filename}.")

In [ ]:
def extract_data(url, method='GET'):
    if method.upper() == 'POST':
        response = requests.post(url)
    else:
        response = requests.get(url)
    
    if response.status_code in [200, 201]:
        data = response.json()
        # Get the URL path without the domain, remove leading/trailing '/'
        path = urlparse(url).path.strip('/')
        # Replace '/' with '_' to create a valid filename
        filename = path.replace('/', '_') + '.json'
        save_data(data, filename)

        metadata = {
            "source": url,
            "request_status": response.status_code,
            "timestamp": datetime.datetime.now().isoformat(),
            "unix_timestamp": int(datetime.datetime.now().timestamp()),
            "hash": hashlib.md5(json.dumps(data).encode("utf-8")).hexdigest()
        }

        save_data(metadata, f'metadata-{filename}')
    else:
        print(f"Failed to extract data from {url}. Status code: {response.status_code}")

In [19]:
for url in input_url:
    extract_data(url)

Data saved to postgres2_ultimas-x-horas-nuevo_24.json.
Data saved to metadata-postgres2_ultimas-x-horas-nuevo_24.json.json.
Data saved to postgres2_datos.json.
Data saved to metadata-postgres2_datos.json.json.


In [20]:
for url in post_url:
    extract_data(url, method='POST')

Data saved to postgres2_datos-diarios-nuevo-2024_7.json.
Data saved to metadata-postgres2_datos-diarios-nuevo-2024_7.json.json.
